$\bullet$ v_fluc is w_fluc

In [1]:
import pickle
import gzip
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.animation as animation
from multiprocessing import Pool

from Helper_Code import *

Uploading the Dataset

In [2]:
# uploading the Complete Dataset
DataFrames = UploadDataSet("dataframes_list.pkl.gz")

# uploading the 'u_fluc' and 'w_fluc' Dataset for all 18000 points
TS_wfluc_all = UploadDataSet("wfluc_TS_Allpoints_Full.pkl.gz")
TS_ufluc_all = UploadDataSet("ufluc_TS_Allpoints_Full.pkl.gz")
TS_vorfluc_all=UploadDataSet("vor_fluc_TS_Allpoints_Full.pkl.gz") 

Extracting the Locations (X,Y)

In [3]:
X, Y = IndexToLocation(DataFrames, np.arange(18000))
X = X.values.reshape(150, 120)
Y = Y.values.reshape(150, 120)

The Reference U_0 (Make sure that it is same as the DataPrep Code)

In [4]:
# A reference Velocity
U_0 = DataFrames[800]["Umean"].mean()
print(U_0.round(5))

3.33294


Computing the Mean Squarred Values for u_fluc, w_fluc & u_fluc_w_fluc

In [5]:
# Create a list to store the DataFrames
dataframes_list = []

# Define the columns
columns = ["MS_u_fluc", "MS_w_fluc", "MS_u_fluc_w_fluc", "MS_vorfluc_vorfluc"]

# Create empty DataFrames and append to the list
for _ in range(18000):
    # Each DataFrame will store the Values for each of the 18000 points
    df = pd.DataFrame(columns=columns)
    dataframes_list.append(df)

In [6]:
TS_vorfluc_all

,vor_fluc_0,vor_fluc_1,vor_fluc_2,vor_fluc_3,vor_fluc_4,vor_fluc_5,vor_fluc_6,vor_fluc_7,vor_fluc_8,vor_fluc_9,...,vor_fluc_17990,vor_fluc_17991,vor_fluc_17992,vor_fluc_17993,vor_fluc_17994,vor_fluc_17995,vor_fluc_17996,vor_fluc_17997,vor_fluc_17998,vor_fluc_17999
0,-232.321543,-196.192245,-239.550979,-265.035398,-229.555309,-230.062757,-355.610763,-426.50398,-440.610431,-447.050735,...,1105.112126,1136.578077,1143.197465,993.272743,285.409852,51.474935,-225.439519,-550.544841,-917.595723,-601.84301
1,-672.957543,-449.403245,-386.386979,-237.176398,-187.007309,-261.779757,-324.419763,-222.33798,-172.857431,-158.053735,...,-25.518874,-20.864923,-151.170535,178.098443,91.527362,-123.657865,-320.573519,-292.594841,-739.582723,-803.63701
2,-155.561043,-241.267245,-279.504979,-265.120398,-295.039309,-360.241757,-336.550763,-323.58098,-287.066431,-206.439735,...,-918.297874,-1136.431923,-1147.182535,-1498.344257,-1603.042148,-664.023865,-280.516519,-1084.407841,-887.105723,-623.57901
3,-158.587543,-177.663245,-241.730979,-220.376398,-213.326309,-114.106357,51.924137,143.58902,179.071569,242.986265,...,-734.252874,-772.671923,-726.733535,-761.117257,-733.942148,-891.076865,-991.364519,-783.892841,-832.928723,-800.14701
4,292.984457,150.721755,52.902021,22.141802,44.804591,154.553243,155.350237,97.43702,115.846569,139.689265,...,-483.651874,-775.706923,-989.742535,-1041.704257,-842.033148,-532.715865,84.634081,41.200159,167.062187,358.92699
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
795,215.721457,103.500755,49.057721,0.788202,-16.876009,-104.488757,-140.265763,-119.55128,-175.666431,-195.592735,...,-618.781874,-679.110923,-715.333535,-586.754257,-669.594148,-488.650865,-204.263519,-98.771841,309.206277,1090.71699
796,-293.772543,-88.063845,-3.423779,16.224902,-106.664609,-174.168757,-125.111063,-94.20428,-63.939131,6.156465,...,-167.693874,-152.683923,-149.993535,-171.581257,-244.177148,-325.987865,-273.512519,-78.083841,-43.443723,46.26199
797,-194.575543,-222.044245,-260.246979,-320.251398,-298.479309,-236.360757,-199.732763,-225.59498,-195.348431,-186.865735,...,-140.232874,-93.398923,-237.531535,-281.332257,-303.470148,-269.873865,-224.176519,-41.946841,-20.253723,101.58809
798,-305.122543,-237.555245,-244.766979,-288.647398,-185.215309,-84.862757,-121.470163,-186.33798,-202.442431,-197.272735,...,-820.070874,-895.640923,-552.882535,-377.705257,-225.606148,-54.263865,-26.260519,-95.590841,-32.177723,84.33099


In [7]:
# alpha belongs to [0, 1]

def process_alpha(alpha, i):
    return Calculate_MS_uu_ww_uw_vorfluc_vorfluc(TS_ufluc_all[f"ufluc_{i}"],
                                  TS_wfluc_all[f"wfluc_{i}"], TS_vorfluc_all[f"vor_fluc_{i}"],
                                  U_0 * alpha,
                                  index=i)

if __name__ == '__main__':
    pool = Pool()

    for i in range(18000):
        df = dataframes_list[i]
        results = pool.starmap(process_alpha, [(alpha, i) for alpha in np.arange(0, 1.01, 0.01)])
        temp = 0
        for val in results:
            df.loc[temp] = np.array(val)
            temp += 1

    pool.close()
    pool.join()

In [8]:
# Define the filename for saving
filename = 'MS_u_fluc__w_fluc__u_fluc_w_fluc__vor_fluc.pkl.gz'

# Open the file using gzip and save the concatenated DataFrame
with gzip.open(filename, 'wb') as f:
    pickle.dump(dataframes_list, f)


In [9]:
loaded_df = UploadDataSet(base_directory="MS_u_fluc__w_fluc__u_fluc_w_fluc__vor_fluc.pkl.gz")
loaded_df[9]

,MS_u_fluc,MS_w_fluc,MS_u_fluc_w_fluc,MS_vorfluc_vorfluc
0,0.303326,0.027489,0.002008,120588.423692
1,0.330914,0.042585,0.002627,137211.150398
2,0.420922,0.079460,0.002880,186902.405333
3,0.566904,0.142143,0.006480,269831.346486
4,0.715002,0.206750,0.006954,347387.910754
...,...,...,...,...
96,NaN,NaN,NaN,NaN
97,NaN,NaN,NaN,NaN
98,NaN,NaN,NaN,NaN
99,NaN,NaN,NaN,NaN
